In [ ]:
!pip install vowpalwabbit
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 16.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=9c4a43d9c15ad1855fb056747770a01e2eac6435e66a94f68097cd997cb509ed
  Stored in directory: /root/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [ ]:
import pandas as pd
import sklearn
import sklearn.model_selection
import sklearn.datasets
import vowpalwabbit

In [ ]:
iris_dataset = sklearn.datasets.load_iris()
iris_dataframe = pd.DataFrame(
    data=iris_dataset.data, columns=iris_dataset.feature_names)
# vw expects labels starting from 1
iris_dataframe["y"] = iris_dataset.target + 1
training_data, testing_data = sklearn.model_selection.train_test_split(
    iris_dataframe, test_size=0.2)

In [ ]:
def to_vw_format(row):
    res = f'{int(row.y)} |'
    for idx, value in row.drop(['y']).iteritems():
        feature_name = idx.replace(' ', '_').replace('(', '').replace(')', '')
        res += f' {feature_name}:{value}'
    return res

In [ ]:
for ex in training_data.head(10).apply(to_vw_format, axis=1):
    print(ex)

1 | sepal_length_cm:5.2 sepal_width_cm:3.5 petal_length_cm:1.5 petal_width_cm:0.2
3 | sepal_length_cm:7.2 sepal_width_cm:3.6 petal_length_cm:6.1 petal_width_cm:2.5
3 | sepal_length_cm:7.7 sepal_width_cm:3.8 petal_length_cm:6.7 petal_width_cm:2.2
2 | sepal_length_cm:5.7 sepal_width_cm:2.6 petal_length_cm:3.5 petal_width_cm:1.0
1 | sepal_length_cm:4.4 sepal_width_cm:2.9 petal_length_cm:1.4 petal_width_cm:0.2
3 | sepal_length_cm:5.8 sepal_width_cm:2.7 petal_length_cm:5.1 petal_width_cm:1.9
1 | sepal_length_cm:4.9 sepal_width_cm:3.1 petal_length_cm:1.5 petal_width_cm:0.2
3 | sepal_length_cm:5.7 sepal_width_cm:2.5 petal_length_cm:5.0 petal_width_cm:2.0
3 | sepal_length_cm:6.8 sepal_width_cm:3.2 petal_length_cm:5.9 petal_width_cm:2.3
2 | sepal_length_cm:5.7 sepal_width_cm:2.9 petal_length_cm:4.2 petal_width_cm:1.3


In [ ]:
vw = vowpalwabbit.Workspace("--oaa 3 --quiet")

# learn from training set with multiple passes
for i in range(10):
  for example in training_data.apply(to_vw_format, axis=1):
      vw.learn(example)

# predict from the testing set
predictions = []
for example in testing_data.apply(to_vw_format, axis=1):
    predicted_class = vw.predict(example)
    predictions.append(predicted_class)

In [ ]:
accuracy = len(testing_data[testing_data.y == predictions]) / len(testing_data)

print(f'Model accuracy {accuracy}')

Model accuracy 0.9333333333333333
